CTGAN

In [ ]:
from ctgan import CTGAN
from ctgan import load_demo
from table_evaluator import load_data, TableEvaluator
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Load the datasets one by one based on senario we need to test ood detection
# here we use validation dataset from same dataset but previously model did not seen 
data1=pd.read_csv('val_data.csv')

In [ ]:
# load ood data set related to same data set we genarated via ctgan
# 
synthetic_data_500e=pd.read_csv('ctgan_cicids2017_ood.csv')

In [ ]:
data2= pd.concat([data1,synthetic_data_500e])
print(data2['Label'].value_counts())

In [ ]:
data2.info()

In [ ]:
data2.head()

In [ ]:
labels1 = data2['Label'].values
data2 = data2.drop(columns=['Label'])


In [ ]:
# Replace infinities with NaNs
data2.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill NaNs with column mean
data2.fillna(data2.mean(), inplace=True)

scaler = MinMaxScaler()
data2 = scaler.transform(data2)

In [ ]:
from tensorflow.keras.models import load_model
# Load the saved model
autoencoder = load_model('Best_AE_model.h5') # you need load the relevent model need test ood detection based on saved models from previously

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, precision_recall_curve, auc, f1_score, f1_score

import time

# Start the timer
start_time = time.time()
# Use the autoencoder to make predictions
predictions = autoencoder.predict(data2)

# Calculate the reconstruction error
mse = np.mean(np.power(data2 - predictions, 2), axis=1)

# Define a range of percentiles to test as thresholds
percentiles = np.arange(1, 100, 1)  # Testing percentiles from 1% to 99%

# Initialize variables to store the best metrics and threshold
best_threshold = 0
best_macro_f1 = 0
best_accuracy = 0
best_conf_matrix = None
best_class_report = None
best_roc_auc = 0
best_pr_auc = 0

for p in percentiles:
    # Calculate the threshold for the current percentile
    threshold = np.percentile(mse, p)
    
    # Classify as anomaly if mse > threshold
    predicted_labels = (mse > threshold).astype(int)
    
    # Calculate metrics
    accuracy = accuracy_score(labels1, predicted_labels)
    macro_f1 = f1_score(labels1, predicted_labels, average='macro')
    conf_matrix = confusion_matrix(labels1, predicted_labels)
    class_report = classification_report(labels1, predicted_labels)
    roc_auc = roc_auc_score(labels1, mse)
    precision, recall, _ = precision_recall_curve(labels1, mse)
    pr_auc = auc(recall, precision)
    
    # If the current macro F1-score is better than the best one, update the best metrics
    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        best_threshold = threshold
        best_accuracy = accuracy
        best_conf_matrix = conf_matrix
        best_class_report = class_report
        best_roc_auc = roc_auc
        best_pr_auc = pr_auc

# Output the best results
print(f'Optimal Percentile Threshold: {best_threshold} (percentile: {p}%)')
print(f'Best Macro F1-Score: {best_macro_f1}')
print(f'Accuracy at Best Macro F1-Score: {best_accuracy}')
print('Confusion Matrix at Best Macro F1-Score:')
print(best_conf_matrix)
print('\nClassification Report at Best Macro F1-Score:')
print(best_class_report)
print(f'ROC-AUC at Best Macro F1-Score: {best_roc_auc}')
print(f'PR-AUC at Best Macro F1-Score: {best_pr_auc}')


# End the timer
end_time = time.time()

# Calculate the training time
training_time = end_time - start_time

print(f"Test time: {training_time} seconds")

In [ ]:
from sklearn.metrics import roc_curve, RocCurveDisplay, roc_auc_score

# Assuming mse is already calculated for the reconstruction error
# and y_test contains the true labels

# Compute ROC curve and ROC area
fpr, tpr, _ = roc_curve(labels1, mse)
roc_auc = roc_auc_score(labels1, mse)

# Plot ROC curve
plt.figure(figsize=(8, 6))
RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='Autoencoder').plot()
plt.title(f'Receiver Operating Characteristic (ROC) Curve\nROC-AUC: {roc_auc:.2f}')
plt.show()

In [ ]:
# Latent Space Visualization with PCA
# encoder = Model(inputs=input_layer, outputs=encoded)
encoded_data2 = encoder.predict(data2)

# Normalize the encoded data
scaler = StandardScaler()
encoded_data2_normalized = scaler.fit_transform(encoded_data2)

# Apply PCA
pca = PCA(n_components=2)
encoded_data2_pca = pca.fit_transform(encoded_data2_normalized)

# Plotting
plt.figure(figsize=(8, 6))
plt.scatter(encoded_data2_pca[:, 0], encoded_data2_pca[:, 1], c=labels1, cmap='viridis')
plt.colorbar()
plt.title('Latent Space Visualization using PCA')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.show()

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Step 1: Encode the data into the latent space using the VAE encoder
encoded_X_test = encoder.predict(data2)

# Step 2: Apply t-SNE to the latent representations
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(encoded_X_test)

# Step 3: Plot the t-SNE results
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=labels1, cmap='viridis', s=10)
plt.colorbar(scatter)
plt.title('t-SNE Visualization of VAE Latent Space')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.show()

Reverse diffusion based OOD

In [ ]:
# load ood data set related to same data set we genarated via Reverse diffusion based model
diff_ood=pd.read_csv('ood_samples_reversediff.csv')

In [ ]:
# Load the datasets one by one based on senario we need to test ood detection
# here we use validation dataset from same dataset but previously model did not seen 
data1=pd.read_csv('val_data.csv')

In [ ]:
data2= pd.concat([data1,diff_ood])

print(data2['Label'].value_counts())

In [ ]:
labels1 = data2['Label'].values
data2 = data2.drop(columns=['Label'])

In [ ]:
# Replace infinities with NaNs
data2.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill NaNs with column mean
data2.fillna(data2.mean(), inplace=True)

data2 = scaler.transform(data2)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, precision_recall_curve, auc, f1_score, f1_score

import time

# Start the timer
start_time = time.time()
# Use the autoencoder to make predictions
predictions = autoencoder.predict(data2)

# Calculate the reconstruction error
mse = np.mean(np.power(data2 - predictions, 2), axis=1)

# Define a range of percentiles to test as thresholds
percentiles = np.arange(1, 100, 1)  # Testing percentiles from 1% to 99%

# Initialize variables to store the best metrics and threshold
best_threshold = 0
best_macro_f1 = 0
best_accuracy = 0
best_conf_matrix = None
best_class_report = None
best_roc_auc = 0
best_pr_auc = 0

for p in percentiles:
    # Calculate the threshold for the current percentile
    threshold = np.percentile(mse, p)
    
    # Classify as anomaly if mse > threshold
    predicted_labels = (mse > threshold).astype(int)
    
    # Calculate metrics
    accuracy = accuracy_score(labels1, predicted_labels)
    macro_f1 = f1_score(labels1, predicted_labels, average='macro')
    conf_matrix = confusion_matrix(labels1, predicted_labels)
    class_report = classification_report(labels1, predicted_labels)
    roc_auc = roc_auc_score(labels1, mse)
    precision, recall, _ = precision_recall_curve(labels1, mse)
    pr_auc = auc(recall, precision)
    
    # If the current macro F1-score is better than the best one, update the best metrics
    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        best_threshold = threshold
        best_accuracy = accuracy
        best_conf_matrix = conf_matrix
        best_class_report = class_report
        best_roc_auc = roc_auc
        best_pr_auc = pr_auc

# Output the best results
print(f'Optimal Percentile Threshold: {best_threshold} (percentile: {p}%)')
print(f'Best Macro F1-Score: {best_macro_f1}')
print(f'Accuracy at Best Macro F1-Score: {best_accuracy}')
print('Confusion Matrix at Best Macro F1-Score:')
print(best_conf_matrix)
print('\nClassification Report at Best Macro F1-Score:')
print(best_class_report)
print(f'ROC-AUC at Best Macro F1-Score: {best_roc_auc}')
print(f'PR-AUC at Best Macro F1-Score: {best_pr_auc}')


# End the timer
end_time = time.time()

# Calculate the training time
training_time = end_time - start_time

print(f"Test time: {training_time} seconds")

Poison Attack

In [ ]:
# Load the datasets one by one based on senario we need to test ood detection
# here we use validation dataset from same dataset but previously model did not seen 
data1=pd.read_csv('val_data.csv')

In [ ]:
poison = pd.read_csv('poison.csv') # we genarate poison data using ART and you can find ART code and use it to genarate posion data related to same dataset you use to train the DL model.

In [ ]:
data2= pd.concat([data1,poison])

print(data2['Label'].value_counts())

In [ ]:
labels1 = data2['Label'].values
data2 = data2.drop(columns=['Label'])

In [ ]:
# Replace infinities with NaNs
data2.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill NaNs with column mean
data2.fillna(data2.mean(), inplace=True)

data2 = scaler.transform(data2)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, precision_recall_curve, auc, f1_score



import time

# Start the timer
start_time = time.time()
# Use the autoencoder to make predictions
predictions = autoencoder.predict(data2)

# Calculate the reconstruction error
mse = np.mean(np.power(data2 - predictions, 2), axis=1)

# Define a range of percentiles to test as thresholds
percentiles = np.arange(1, 100, 0.5)  # Testing percentiles from 50% to 99%

# Initialize variables to store the best metrics and threshold
best_threshold = 0
best_f1 = 0
best_accuracy = 0
best_conf_matrix = None
best_class_report = None
best_roc_auc = 0
best_pr_auc = 0

for p in percentiles:
    # Calculate the threshold for the current percentile
    threshold = np.percentile(mse, p)
    
    # Classify as anomaly if mse > threshold
    predicted_labels = (mse > threshold).astype(int)
    
    # Calculate metrics
    accuracy = accuracy_score(labels1, predicted_labels)
    f1 = f1_score(labels1, predicted_labels)
    conf_matrix = confusion_matrix(labels1, predicted_labels)
    class_report = classification_report(labels1, predicted_labels)
    roc_auc = roc_auc_score(labels1, mse)
    precision, recall, _ = precision_recall_curve(labels1, mse)
    pr_auc = auc(recall, precision)
    
    # If the current F1-score is better than the best one, update the best metrics
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold
        best_accuracy = accuracy
        best_conf_matrix = conf_matrix
        best_class_report = class_report
        best_roc_auc = roc_auc
        best_pr_auc = pr_auc

# Output the best results
print(f'Optimal Percentile Threshold: {best_threshold} (percentile: {p}%)')
print(f'Best F1-Score: {best_f1}')
print(f'Accuracy at Best F1-Score: {best_accuracy}')
print('Confusion Matrix at Best F1-Score:')
print(best_conf_matrix)
print('\nClassification Report at Best F1-Score:')
print(best_class_report)
print(f'ROC-AUC at Best F1-Score: {best_roc_auc}')
print(f'PR-AUC at Best F1-Score: {best_pr_auc}')


# End the timer
end_time = time.time()

# Calculate the training time
training_time = end_time - start_time

print(f"Test time: {training_time} seconds")